In [1]:
import os
from collections import defaultdict
import numpy as np

M_ROWS = 3  # строк в A
K_COLS = 7  # столбцов в A (и строк в B)
N_COLS = 3  # столбцов в B

$C_{ij} = ∑A_{ik}*B_{kj}$

In [2]:
def map_function(matrix_name, row_index, line_content):
    values = list(map(float, line_content.strip().split(',')))
    emitted_pairs = []

    if matrix_name == 'A':
        for k, a_val in enumerate(values):
            for j in range(N_COLS):
                key = (row_index, j)
                value = ('A', k, a_val)
                emitted_pairs.append((key, value))
                
    elif matrix_name == 'B':
        k = row_index
        for j, b_val in enumerate(values):
            for i in range(M_ROWS):
                key = (i, j)
                value = ('B', k, b_val)
                emitted_pairs.append((key, value))
                
    return emitted_pairs

def shuffle_function(mapped_data):
    grouped_data = defaultdict(list)
    for key, value in mapped_data:
        grouped_data[key].append(value)
    return grouped_data

def reduce_function(key, values):    
    hash_a = {}
    hash_b = {}
    
    for src, k, val in values:
        if src == 'A':
            hash_a[k] = val
        else:
            hash_b[k] = val
            
    result_val = 0.0
    # сумма по k (от 0 до K-1)
    for k in hash_a:
        if k in hash_b:
            result_val += hash_a[k] * hash_b[k]
            
    return key, result_val

In [3]:
def map_reduce():
    all_mapped_items = []
    
    with open('matrix_a.txt', 'r') as f:
        for i, line in enumerate(f):
            pairs = map_function('A', i, line)
            all_mapped_items.extend(pairs)
            
    with open('matrix_b.txt', 'r') as f:
        for k, line in enumerate(f):
            pairs = map_function('B', k, line)
            all_mapped_items.extend(pairs)


    grouped_data = shuffle_function(all_mapped_items)

    final_matrix = {}
    for key, values in grouped_data.items():
        idx, val = reduce_function(key, values)
        final_matrix[idx] = val
        print(f"Cell {idx}: {val}")

    return final_matrix

In [4]:
result = map_reduce()

Cell (0, 0): 9813.0
Cell (0, 1): 35879.0
Cell (0, 2): 15106.0
Cell (1, 0): -11386.0
Cell (1, 1): -48190.0
Cell (1, 2): -22091.0
Cell (2, 0): -622.0
Cell (2, 1): -3192.0
Cell (2, 2): -20609.0


In [5]:
res = [[_ for x in range(M_ROWS)] for x in range(M_ROWS)]
for key in result:
    res[key[0]][key[1]] = result[key]
res

[[9813.0, 35879.0, 15106.0],
 [-11386.0, -48190.0, -22091.0],
 [-622.0, -3192.0, -20609.0]]

In [6]:
A = np.loadtxt('matrix_a.txt', delimiter=',')
B = np.loadtxt('matrix_b.txt', delimiter=',')
C_numpy = A @ B
C_numpy

array([[  9813.,  35879.,  15106.],
       [-11386., -48190., -22091.],
       [  -622.,  -3192., -20609.]])